In [17]:
import pandas as pd
import json
from tqdm import tqdm
import numpy as np
from scipy.stats import kendalltau
from sklearn.metrics import ndcg_score
from openai import OpenAI

In [18]:
name = "rerank_deepseek_bing.tsv"
res = pd.read_csv(name, sep="\t")
res[res["query-id"] == 0]

,llm-score,human-score,query-id,corpus-id
0,2.0,1.0,0.0,597.0
1,2.0,1.0,0.0,598.0
2,2.0,1.0,0.0,599.0
3,2.0,1.0,0.0,616.0
4,2.0,1.0,0.0,617.0
5,2.0,1.0,0.0,618.0
6,2.0,2.0,0.0,620.0
7,0.0,2.0,0.0,621.0
8,2.0,2.0,0.0,622.0
9,2.0,0.0,0.0,9568.0


In [19]:
e5_results = pd.read_json("results_wikifacts-window_3_v0_dev.json")
def get_top_k_nlargest(df, k=20):
    """
    Использует метод nlargest для каждого столбца
    """
    results = {
        "query-id": [],
        "corpus-id": [],
        "e5-score": [],
    }
    
    for query_id in df.columns:
        top_k = df[query_id].nlargest(k)
        n = len(top_k)
        results["query-id"] += [query_id for _ in range(n)]
        results["corpus-id"] += top_k.index.tolist()
        results["e5-score"] += top_k.values.tolist()
    return pd.DataFrame(results)

e5_df = get_top_k_nlargest(e5_results)

In [20]:
merged = pd.merge(
        e5_df,
        res,
        on=['query-id', 'corpus-id'],
        how='left'
    )
del merged["e5-score"]
merged[merged["query-id"] == 0]

,query-id,corpus-id,llm-score,human-score
0,0,599,2.0,1.0
1,0,91397,2.0,0.0
2,0,140938,2.0,0.0
3,0,9725,2.0,0.0
4,0,598,2.0,1.0
5,0,9568,2.0,0.0
6,0,140781,1.0,0.0
7,0,91240,2.0,0.0
8,0,616,2.0,1.0
9,0,618,2.0,1.0


In [21]:
len(merged)

10680

In [22]:
merged.to_csv(name, sep='\t', index=False)